In [ ]:
df = spark.read.parquet("gs://bbm-labs-ds-news/bbm-news-with-content.parquet")
# print df.count()

In [10]:
# insert into elastic search
df.write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes","elasticsearch-1-vm,elasticsearch-2-vm, elasticsearch-3-vm")\
    .option("es.resource","bbmnews/article")\
    .option("es.mapping.id", "article_id")\
    .save()

In [4]:
# df.show(1)

+--------------------+--------------------+--------------+------------+----------+-----------+--------------------+--------------------+--------------------+--------+
|                date|         article_url|publisher_name|channel_name|article_id|total_views|               title|       image_caption|             content|language|
+--------------------+--------------------+--------------+------------+----------+-----------+--------------------+--------------------+--------------------+--------+
|2017-08-11 00:00:...|https://news.bbme...|  Liputan6.com|        Home|    232938|          3|5 Resep Praktis B...|5 resep jajanan i...|Membawakan bekal ...|      id|
+--------------------+--------------------+--------------+------------+----------+-----------+--------------------+--------------------+--------------------+--------+
only showing top 1 row



In [29]:
import json

# query from elastic search
q ="""{ 
        "min_score": 0.7,
        "query":{
            "dis_max": {
                "queries":[
                {
                    "multi_match" : {
                        "fields":["article_url","title","image_caption","content"],
                        "query" : "makan makanan masak Kater resep tukang roti kue teh curry arsik hotel kafe restoran goreng nasi sate sambal ayam mie",
                        "type":"most_fields",
                        "analyzer": "standard"
                        }
                    
                },
                {
                    "query_string": {
                            "fields":["article_url","title","image_caption","content"],
                            "query": "rumah makan OR juru masak"        
                            }
                }
            ]
        }
    }
}"""

es_read_conf = {
    "es.nodes" : "elasticsearch-1-vm",
    "es.port" : "9200",
    "es.resource" : "bbmnews/article",
    "es.query" : q,
    "es.read.metadata": "true"
}

es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf=es_read_conf)

new_df = sqlContext.createDataFrame(es_rdd)

In [30]:
new_df.count()

41955

In [16]:
new_df.show(1,False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [92]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import col

keys = (new_df
    .select(explode("paramaters"))
    .select("key")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect())

In [94]:
exprs = [col("paramaters").getItem(k).alias(k) for k in keys]
data = new_df.select(*exprs)

In [96]:
data.count()

41955

In [109]:
data.write.parquet("gs://url_catag_data/data_process_code/results/es_res.parquet")

In [110]:
testing = spark.read.parquet("gs://url_catag_data/data_process_code/results/es_res.parquet")

In [111]:
testing.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------+-----------------+------------+-----------+--------------------+
|               title|             content|article_id|       image_caption|                date|language|   publisher_name|channel_name|total_views|         article_url|
+--------------------+--------------------+----------+--------------------+--------------------+--------+-----------------+------------+-----------+--------------------+
|FOTO: Yuk Intip B...|Mel Gibson baru s...|    269417|FOTO: Yuk Intip B...|2017-08-09 00:00:...|      id|        Kapanlagi|     Hiburan|         89|https://news.bbme...|
|Liga 2: Melcior H...|PSIS Semarang pes...|    284146|Partai PPSM vs PS...|2017-08-16 00:00:...|      id|         Bola.com|       Sport|         84|https://news.bbme...|
|Untuk Kali Kedua,...|











 Untu...|    143511|Salah satu penyer...|2017-07-16 00:00:...|      id|        Juara.net|       Sport|          3|ht